# Interview agent using langchain

## Step 1 - the dependencies

In [1]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.tools import tool
from pydantic import BaseModel, Field
from typing import List, Literal
import uuid

# agents
from langchain.agents import create_agent
from langchain_openai import AzureChatOpenAI
import json

/home/anjanavaasu2404/anaconda3/envs/training_env/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


## Step 2 - tools and agents

In [2]:
# document loader tool (import JD locally)

@tool
def load_JD(jd_path:str):
    """Loads job description parseable for LLM
    Args:
        jd_path (str): local path to Job description
    Returns:
        jd (str): Job description
    """
    # initialize loader
    loader = UnstructuredMarkdownLoader(jd_path,
                    mode='single',
                    strategy='fast')
    
    docs = loader.load()
    return docs[0].page_content


In [ ]:
# define memory components - store the interview QA with JD here
class InterviewQA(BaseModel):
    """QA pair for logging """
    question_id: uuid.UUID = Field(default_factory=uuid.uuid4)
    question: str | None = None
    expected_answer: str | None = None
    answer: str | None = None
    difficulty_level: Literal["easy", "medium", "hard"] = "easy" # default values
    score: int | None = None
    justification: str | None = None


class InterviewState(BaseModel):
    """Stores interview records"""
    record_id: uuid.UUID = Field(default_factory=uuid.uuid4)
    jd_path: str
    role: str | None = None
    jd: str | None = None
    field: str | None = None
    interview_questions: List[InterviewQA] = []
    decision: Literal["hire", "no-hire"] = "no-hire"
    reason: str | None = None

# below classes are for validating model outputs
# Future to-do: refactor them
class InterviewQAPairs(BaseModel):
    """For generating questions"""
    question_id: uuid.UUID = Field(default_factory=uuid.uuid4)
    question: str
    expected_answer: str
    difficulty_level: Literal["easy", "medium", "hard"]

# for output validation from JD analyzing
class QAoutputpairs(BaseModel):
    """Validating output from JD understadning"""
    role: str
    jd: str
    qa_pairs: List[InterviewQAPairs]

class Evaluation(BaseModel):
    question_id: str
    score: int = Field(ge=1, le=10)
    justification: str

# for output validation from candidate response evaluation
class EvaluationDecision(BaseModel):
    """Validating decision agent output"""
    interview_questions: List[Evaluation]
    decision: Literal["hire", "no-hire"]
    reason: str


In [ ]:
# model setup
import os
from openai import AzureOpenAI

AZURE_OPENAI_ENDPOINT = os.getenv(
    "AZURE_OPENAI_ENDPOINT", ""
)

# REMOVE KEY
AZURE_OPENAI_API_KEY = os.environ.get(
    "AZURE_OPENAI_API_KEY",
    "",
)
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")

if not AZURE_OPENAI_API_KEY:
    raise ValueError(
        "Missing AZURE_OPENAI_API_KEY env var. Set it securely (do not hardcode keys)."
    )

CHAT_DEPLOYMENT = "gpt-4.1-mini"

# client = AzureOpenAI(
#     api_version=AZURE_OPENAI_API_VERSION,
#     azure_endpoint=AZURE_OPENAI_ENDPOINT,
#     api_key=AZURE_OPENAI_API_KEY,
# )

model = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    deployment_name=CHAT_DEPLOYMENT,
    temperature=0.2,
)

print("AzureOpenAI client ready")

AzureOpenAI client ready


In [5]:
# get the agents
jd_understanding_agent = create_agent(
    model,
    tools = [load_JD],
    system_prompt=("""You are an interviewer with expertise in your field. 
                   Load the job description by calling the tool `load_JD`.
                   Using the job description from above, identify the role, field from the job description provided and generate upto six questions to be used to interview a candidate.
                   Return only valid JSON matching this schema:
                   {{"role", string,
                   "field": string
                   "jd": string - provide summary of the job description here,
                   "qa_pairs": [
                                {{
                                    "question": "string",
                                    "expected_answer": "string",
                                    "difficulty_level": "easy | medium | hard"}}
                                ]
                   }}""")
)

decision_agent = create_agent(
    model,
    system_prompt=("""You are an interviewer with expertise in your field incharge of hiring a candidate. 
                   You will be given the role, field and interview questions, which contain:
                        - question id
                        - questions asked
                        - question difficulty level
                        - candidate answer 
                        - expected answer for the question
                   Your task is to:
                   1. Provide a score between 1 to 10 to each of the candidate responses based on the expected answer
                   2. Provide reasoning for the score provided
                   3. Based on the scores and justification, decide whether the candidate is suitable for the role or not. If the candidate is suitable, respond with hire, else respond with no-hire.
                   4. Justify the hiring decision concisely, by reviewing the question level, scores and reasons for the scores given to the answers.
                   Return only valid JSON matching this schema:
                   {{
                    "interview_questions": [{{
                            "question_id": string,
                            "score": number between 1 to 10,
                            "justification": string, reason for the score provided in 100 words
                   }}],
                    "decision": "hire | no-hire",
                   "reason": "string, provide reason for hiring based on the evaluation above in 100 words"
                   }}""")
)

## Nodes

In [7]:
def generate_questions_from_jd(state: InterviewState) -> InterviewState:
    """Generates interview questions after understanding job description
    Args:
        state: state of the node
    Returns:
        state: modified state with the interview questions"""
    # Agent call
    response = jd_understanding_agent.invoke({"messages": [{
        "role": "user", 
        "content": f"Use the path provided here to fetch the job description: {state.jd_path}"}]})

    # Validate strictly
    # print(response['messages'][-1].content)
    # print(type(response['messages'][-1].content))
    parsed = QAoutputpairs.model_validate_json(response['messages'][-1].content)

    state.interview_questions = [InterviewQA(question_id = qa.question_id,
                                             question = qa.question,
                                             expected_answer = qa.expected_answer,
                                             difficulty_level=qa.difficulty_level)
                                for qa in parsed.qa_pairs]
    return state

In [8]:
def answer_interview(state: InterviewState) -> InterviewState:
    """Conduct the interview based on the questions"""

    for i, question in enumerate(state.interview_questions):
        print(f"\n Question {i+1}:\n{question.question}\n")
        answer = input("Your answer: ").strip()
        question.answer = answer
    
    print(f"Completed the interview successfully! Please wait for a few moments to get the results")
    return state

In [9]:
def review_interview(state: InterviewState) -> InterviewState:
    """Review the transcript provided"""

    # prepare the messages:
    interview_questions = []
    for qa in state.interview_questions:
        interview_questions.append({
            "question_id": str(qa.question_id),
            "question": qa.question,
            "difficulty_level": qa.difficulty_level,
            "expected_answer": qa.expected_answer,
            "candidate_answer": qa.answer
        })
    user_msg = f"""role: {state.role}\nfield: {state.field}\nInterview questions:{'-' * len('Interview questions')}\n
                    {json.dumps(interview_questions)}"""
    
    print(f"Awaiting interview results.....")
    response = decision_agent.invoke({"messages": [{"role": "user", "content": user_msg}]})
    print(f"\nEvaluation results: {response["messages"][-1].content}")

    # validation and assign to the state
    parsed = EvaluationDecision.model_validate_json(response["messages"][-1].content)

    # assign to the state
    score_map = {
        item.question_id: item
        for item in parsed.interview_questions
    }

    for qa in state.interview_questions:
        eval_item = score_map.get(str(qa.question_id))
        if eval_item:
            qa.score = eval_item.score
            qa.justification = eval_item.justification
    
    return state
    

## Compiling the workflow

In [10]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import InMemorySaver

checkpoint = InMemorySaver()
graph = StateGraph(InterviewState)

graph.add_node(
    "generate_questions",
    generate_questions_from_jd
)
graph.add_node(
    "ask_questions",
    answer_interview
)

graph.add_node(
    "evaluate_candidate",
    review_interview
)

graph.set_entry_point("generate_questions")
graph.set_finish_point("evaluate_candidate")

graph.add_edge("generate_questions", "ask_questions")
graph.add_edge("ask_questions", "evaluate_candidate")


In [12]:
app = graph.compile(checkpointer=checkpoint)
jd_1 = InterviewState(jd_path="data/cloud_engineering_manager_lafosse.md")
final_state = app.invoke(jd_1, config={"configurable": {"thread_id": jd_1.record_id}})


 Question 1:
Can you describe your experience with Azure Landing Zones and how you have implemented them in previous projects?


 Question 2:
How do you approach automating cloud infrastructure using Infrastructure as Code tools like Terraform or ARM templates?


 Question 3:
Explain how you have incorporated DevOps philosophies and culture into your teams or projects.


 Question 4:
What strategies do you use to ensure cloud security, particularly with Azure Active Directory and network connectivity?


 Question 5:
Describe a complex technical or operational problem you have solved in a cloud environment and how you approached it.


 Question 6:
How do you balance the use of agile and waterfall methodologies in managing cloud engineering projects?

Completed the interview successfully! Please wait for a few moments to get the results
Awaiting interview results.....

Evaluation results: {
  "interview_questions": [
    {
      "question_id": "9966d6cd-e06c-4bc7-974f-5c3c5e927d27",
   

In [13]:
# get individual scoring
for question in final_state['interview_questions']:
    print(f"Question: {question.question}\n\nAnswer: {question.answer} \n\nScore: {question.score}\n\nJustification: {question.justification}\n\n")

Question: Can you describe your experience with Azure Landing Zones and how you have implemented them in previous projects?

Answer: My experience centers on the Cloud Adoption Framework (CAF). I view a Landing Zone not just as a subscription, but as a multi-subscription environment that handles scale, security, governance, and networking upfront.  Key Implementation Pillars In previous deployments, I have focused on these four critical areas:  Enterprise Enrollment & Identity: Setting up the Azure AD (Entra ID) tenant structure and implementing RBAC (Role-Based Access Control) using the principle of least privilege.  Resource Organization: Establishing a Management Group hierarchy (e.g., Platform vs. Landing Zones) to allow for policy inheritance and granular control.  Network Topology: Implementing a Hub-and-Spoke architecture.  Hub: Contains shared services like Azure Firewall, VPN/ExpressRoute gateways, and Central Logging (Log Analytics).  Spoke: Isolated subscriptions for specifi

In [26]:
config = {"configurable": {"thread_id": jd_1.record_id}}

# Retrieve the state
stored_state = app.get_state(config)
print(stored_state)

StateSnapshot(values={'record_id': UUID('0a50284b-feaf-4630-aa3d-0e39b85daf38'), 'jd_path': 'data/cloud_engineering_manager_lafosse.md', 'interview_questions': [InterviewQA(question_id=UUID('9966d6cd-e06c-4bc7-974f-5c3c5e927d27'), question='Can you describe your experience with Azure Landing Zones and how you have implemented them in previous projects?', expected_answer='The candidate should explain their hands-on experience designing and deploying Azure Landing Zones, including how they structured environments for scalability, security, and compliance, and any challenges faced during implementation.', answer='My experience centers on the Cloud Adoption Framework (CAF). I view a Landing Zone not just as a subscription, but as a multi-subscription environment that handles scale, security, governance, and networking upfront.  Key Implementation Pillars In previous deployments, I have focused on these four critical areas:  Enterprise Enrollment & Identity: Setting up the Azure AD (Entra I

In [27]:
# next role: SDE2:
jd_2 = InterviewState(jd_path="data/SDE2_jd.md")
final_state = app.invoke(jd_1, config={"configurable": {"thread_id": jd_2.record_id}})


 Question 1:
Can you describe your experience with Azure Landing Zones and how you have implemented them in previous projects?


 Question 2:
How do you approach automating cloud infrastructure using Infrastructure as Code tools like Terraform or ARM templates?


 Question 3:
Explain your experience with Azure Active Directory, particularly in managing identities, roles, and security groups.


 Question 4:
What strategies do you use to ensure cloud platform security and connectivity, including network configurations like vNet, firewalls, and VPNs?


 Question 5:
Describe your experience with DevOps practices and tools, including how you have implemented deployment pipelines and managed git repositories.


 Question 6:
How do you balance technical leadership with managing cross-functional teams and influencing decision makers?

Completed the interview successfully! Please wait for a few moments to get the results
Awaiting interview results.....

Evaluation results: {
  "interview_quest

In [28]:
# next role: SDE2:
jd_3 = InterviewState(jd_path="data/SDE2_jd.md")
final_state = app.invoke(jd_3, config={"configurable": {"thread_id": jd_3.record_id}})


 Question 1:
Can you explain the importance of lower-level design in software development and how you ensure code quality during implementation?


 Question 2:
Describe a complex problem you solved using data structures and algorithms. What approach did you take and why?


 Question 3:
How do you approach performance optimization and scalability in software applications?


 Question 4:
What experience do you have with version control systems like Git, and how do you use them in a collaborative environment?


 Question 5:
Can you discuss your experience working in startup environments and how it has influenced your software development approach?


 Question 6:
Explain how you participate in architectural and system design discussions. What key factors do you consider?

Completed the interview successfully! Please wait for a few moments to get the results
Awaiting interview results.....

Evaluation results: {
  "interview_questions": [
    {
      "question_id": "b83fe90f-b546-437f-8c7d

In [29]:
# get individual scoring
for question in final_state['interview_questions']:
    print(f"Question: {question.question}\n\nAnswer: {question.answer} \n\nScore: {question.score}\n\nJustification: {question.justification}\n\n")

Question: Can you explain the importance of lower-level design in software development and how you ensure code quality during implementation?

Answer: Lower-level design is critical because it translates high-level architecture into concrete, implementable components—classes, interfaces, data structures, and control flows. It’s where decisions around modularity, extensibility, and performance actually get locked in. A strong LLD reduces ambiguity for developers, minimizes rework, and makes the system easier to test, debug, and evolve over time.  To ensure code quality during implementation, I focus on a few key practices. First, I design with clear responsibilities using principles like SOLID and separation of concerns. Second, I write code that’s testable by design—using dependency injection and meaningful interfaces—so unit tests are easy to add and maintain. I also rely on code reviews to catch design and readability issues early, and use static analysis, linters, and consistent cod